In [1]:
from tqdm import tqdm
from BurstAdmaDatasetLoader import BurstAdmaDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split
import json 
import time
from graphs.recurrent.graphs_evolvegcn_h_improved import ModelOps
import os
import torch
import torch.multiprocessing as mp
import json
import time
import pandas as pd

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

MAX_PROCESS = 1 #int(mp.cpu_count()*0.7)
num_each_threads = 1 # int(mp.cpu_count()*0.7/MAX_PROCESS) 
torch.set_num_threads(num_each_threads)
torch.set_num_interop_threads(num_each_threads)
 
  
t = int(time.time())
g_ml = None
def execute_one(loader, lags, train_ratio, num_train,current_try, lr):

    # torch.set_num_threads(num_each_threads)
    # torch.set_num_interop_threads(num_each_threads)

    dataset = loader.get_dataset(lags=lags)
    # device = torch.device('cuda')
    # dataset = dataset.to(device)
    train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=train_ratio)
    ml_structure = ModelOps(lr=lr)
    global g_ml
    g_ml = ml_structure
    ml_structure.train(loader, train_dataset, num_train=num_train, plot_model=False,
                       calc_perf=True)
    metrics = ml_structure.eval(test_dataset, plot_model=False)
    metrics["nt"] = num_train
    metrics["tr"] = train_ratio
    metrics["l"] = lags
    metrics["lr"] = lr
    # metrics["model"] = ml_structure 
    # if metrics['p']>0.99 or metrics['r']>0.99 or metrics['f']>0.99  or  metrics['a']>0.99  or   metrics['m']>0.99  :
    # torch.save(ml_structure.model, f"./runs/{t}/saved_model_{current_try}_{lr}_{lags}_{train_ratio}_{num_train }")
    # ml_structure.history1.progress()
    # ml_structure.history1.save(f"./runs/{t}/saved_log_{current_try}_{lr}_{lags}_{train_ratio}_{num_train }.pkl")
    # ml_structure.plot(["a","p","r","f"])
    # ml_structure.save_after_plot(f"split1/saved_plot_4_{current_try}_{lags}_{train_ratio}_{num_train }.png")
    # ml_structure.plot(["a","p","r","f","m"])
    # ml_structure.save_after_plot(f"split1/saved_plot_5_{current_try}_{lags}_{train_ratio}_{num_train }.png")

    metrics["current_try"] = current_try
    # q.put(metrics)
    return metrics



loader = BurstAdmaDatasetLoader(negative_edge=False,features_as_self_edge=True)  # , negative_edge=True)
LAG = 1
TRAIN_RATION = 0.7
NUM_TRAIN = 2
LEARNING_RATE = 0.0009

metrics = execute_one(loader, LAG, TRAIN_RATION, NUM_TRAIN,NUM_TRAIN,LEARNING_RATE)
 
print(metrics)



{'p': 0.0, 'r': 0.0, 'f': 0.0, 'a': 0.961917808219177, 'a1': 0.9691917808219144, 'm': 0.2805773128360005, 'c': 1.4638537168502808, 'nt': 2, 'tr': 0.7, 'l': 1, 'lr': 0.0009, 'current_try': 2}


In [1]:
print(g_ml)

NameError: name 'g_ml' is not defined